In [1]:
!pip install pmaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pmaw import PushshiftAPI
import datetime as dt

from time import sleep
import requests
import pandas as pd

In [3]:
# Extração local no subreddit do país

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='chile',
                               after = start_time, 
                               before = end_time, 
                               q = '"coronavirus"||"covid"||"covid19"||"covid-19"||"corona virus"||"sars cov"||"sarscov"||"sars-cov-2"',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list_local = [post for post in posts]

df_post = pd.json_normalize(post_list_local)
print(df_post['id'].shape)

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('chile_local.csv')

(720,)


In [6]:
# Extração geral no subreddit Coronavirus

start_time = int(dt.datetime(2020, 1, 1).timestamp())  
end_time = int(dt.datetime(2020, 7, 31).timestamp())   

api = PushshiftAPI()
posts = api.search_submissions(subreddit='Coronavirus',
                               after = start_time, 
                               before = end_time, 
                               q = 'chile',
                               limit = None,
                               fields = ['id', 'author', 'created_utc', 'domain', 'url', 'title', 'num_comments', 'full_link', 'permalink'])
post_list_global = [post for post in posts]

df_post = pd.json_normalize(post_list_global)
print(df_post['id'].shape)

comments = []
i = 1
for post in df_post['id'].values:
  r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
  while r.status_code != 200:
    sleep(min((2**i), 60))
    r = requests.get(f"https://api.pushshift.io/reddit/comment/search?q=*&link_id={post}&fields=author,body,created_utc,id,parent_id&limit=1000")
    i += 1

  i = 1
  comments.append(r.json())

pd.json_normalize(pd.json_normalize(comments).explode('data')['data']).to_csv('chile_global.csv')

(142,)


In [9]:
pd.read_csv('/content/chile_global.csv')

,Unnamed: 0,author,body,created_utc,id,parent_id
0,0,KMiller152,\*gets killed by pacos\*,1.583331e+09,fjgnbj0,t1_fjgk622
1,1,isaac_horstmeier,Hold my hand papurri papà,1.583329e+09,fjgkofr,t1_fjgk622
2,2,DaydreamBliss,I don't wanna die mister Piñera.,1.583329e+09,fjgk622,t3_fdcxxn
3,3,cl0udNinja,"The man was discharged, as he was in stable co...",1.583329e+09,fjgjzr5,t3_fdcxxn
4,4,AutoModerator,Welcome to r/Coronavirus! We have a very speci...,1.583328e+09,fjgjarr,t3_fdcxxn
...,...,...,...,...,...,...
1135,1135,tradonis,Why? It's the same idea how you count flu deat...,1.592686e+09,fvh2gh6,t1_fvgy9ke
1136,1136,Callec254,That seems... dishonest.,1.592684e+09,fvgy9ke,t3_hcsjuo
1137,1137,AutoModerator,Automatic translation [here](https://translate...,1.592683e+09,fvgwhye,t3_hcsjuo
1138,1138,NaN,NaN,NaN,NaN,NaN


In [10]:
pd.read_csv('/content/chile_local.csv')

,Unnamed: 0,author,body,created_utc,id,parent_id
0,0,Tukurito,El clásico beatle está de vuelta en Patronato.,1.581608e+09,fhhmlfq,t3_f292cd
1,1,aanl01,Ahh. En todo caso ojo por 3 cosas: en primer l...,1.581522e+09,fheg24d,t1_fhe6mnn
2,2,tolar777,[THIS](https://images.ontheedgebrands.com/cdn-...,1.581519e+09,fhebghi,t3_f292cd
3,3,PossiblyAKnob,Los 18300 son los muertos confirmados por labo...,1.581515e+09,fhe6mnn,t1_fhe0fz9
4,4,aanl01,"Fake news. En la pandemia de 2009 ""solo"" murie...",1.581510e+09,fhe0fz9,t1_fhb6k2g
...,...,...,...,...,...,...
18881,18881,Javiercdx,"Creo que la opción B es más bien dañina, puest...",1.585169e+09,flhs77h,t1_flhru4j
18882,18882,TastyZero,A estas alturas creo que es una mezcla de: \n...,1.585168e+09,flhru4j,t3_foxctm
18883,18883,Javiercdx,"Ya pero, esa página recopila información que e...",1.585168e+09,flhrq4j,t1_flhrd4m
18884,18884,juancprieto,https://www.worldometers.info/coronavirus/\n\n...,1.585168e+09,flhrd4m,t3_foxctm
